In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import glob
from sklearn import model_selection
from config import Config
import utils
import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()
os.chdir(ROOT_DIR)

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to COCO trained weights
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

MODEL_PATH = ""

In [ ]:
class irisConfig(Config):
    
    
    # Give the configuration a recognizable name
    NAME = "irises"

    GPU_COUNT = 1
    IMAGES_PER_GPU = 2

    NUM_CLASSES = 1 + 1  # background + 3 shapes

    IMAGE_MIN_DIM = 768
    IMAGE_MAX_DIM = 1024

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 1000

    VALIDATION_STEPS = 50
    


class InferenceConfig(irisConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
inference_config.display()
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

#Loads weights from a static model file
model.load_weights(MODEL_PATH, by_name=True)

In [ ]:
import matplotlib.pyplot as plt
import os
import glob
import cv2
from sklearn import model_selection
import re
from scipy import ndimage
import skimage.io
from PIL import Image
import numpy as np
from skimage.io import imread
from skimage.color import gray2rgb
import time
from visualize import display_images
from visualize import display_weight_stats

class_names = ['BG','iris']

# Root directory of the project

# Load images from folder

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

#Inference one at a time, change to batch (Memory per image calculate)
def runonFolder(inputFolder,saveFolder):
    os.chdir(folderPath)
    imList = sorted(glob.glob("*"),key=numericalSort)
    print (  len(imList))
    avgTime = 0.0
    
    newpath = saveFolder
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    os.chdir(newpath)
    start = time.time()

    print ("Saving in " + os.getcwd())
    for i in range(len(imList)):
        im = imread(folderPath + "/" + imList[i])
        image = skimage.color.gray2rgb(im)
        im, window, scale, padding, crop = utils.resize_image(
        image,
        min_dim=inference_config.IMAGE_MIN_DIM,
        min_scale=inference_config.IMAGE_MIN_SCALE,
        max_dim=inference_config.IMAGE_MAX_DIM,
        mode="pad64")

        results = model.detect([im], verbose=0)
        r = results[0]
        mask = r['masks'][:, :, 0]
        end = time.time()
        avgTime += (end-start)

        print (end-start)

        saveName = imList[i]

        saveName = re.sub('[.].*','.png',saveName)
        visualize.display_instances(im, r['rois'], r['masks'], r['class_ids'],class_names,show_bbox = True, ax=get_ax())
        matplotlib.image.imsave(saveName, mask)            

    print (avgTime)

#Folder containing iris images
folderPath = ROOT_DIR+"/ub"

#Segmented masks stored in /segmented
runonFolder(folderPath,ROOT_DIR+"/segmented")

